<a href="https://colab.research.google.com/github/pablobermudez/Up/blob/main/HPC/Ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Introducción

Este ejercicio poseerá una multiplicación de matrices realizada con dos implementaciones diferentes: la primera utilizando solamente el lenguaje Python y la segunda haciendo uso del GPU con el lenguaje Python y Cuda junto con el framework pycuda.

Su finalidad, además de demostrarnos las diferencias en los tiempos de ejecución según el tamaño de las matrices ingresadas, será el der comprender como Cuda hace uso del manejo de su ejecución en paralelo de hilos para modificar y simplificar lo que sería un algoritmo de multiplicación de matrices como lo solemos hacer en otro tipo de lenguaje.


#2 Armado del ambiente

Instalar en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

#3 Desarrollo

##3.1 Desarrollo - CPU

In [7]:
# --------------------------------------------
#@title 3.1.1 Parámetros de ejecución { vertical-output: true }

Filas_Matriz_A =   2#@param {type: "number"}
Columnas_Matriz_B = 3#@param {type: "number"}
Filas_B_Columnas_A = 1#@param {type: "number"}
# --------------------------------------------

# --------------------------------------------
# Valido número de ingreso por el usuario
if (Filas_B_Columnas_A <= 0 or Filas_Matriz_A < 0 or Columnas_Matriz_B < 0 ):
  raise Exception("Por favor, no ingrese números negativos. El atributo Filas_B_Columnas_A no puede ser igual a 0") 
# --------------------------------------------

import numpy as np
from datetime import datetime
tiempo_total = datetime.now()


# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

# --------------------------------------------

# CPU - Genero las matrices segun los parametros
X = np.random.randint(0, 100, (Filas_Matriz_A, Filas_B_Columnas_A))
Y = np.random.randint(0, 100, (Filas_B_Columnas_A, Columnas_Matriz_B))
result = np.zeros((Filas_Matriz_A,Columnas_Matriz_B), dtype=int)

for i in range(len(X)):
   for j in range(len(Y[0])):
       for k in range(len(Y)):
           result[i][j] += X[i][k] * Y[k][j]

print( "Resultados:" )
print( "------------------------------------")
print("Matriz X: ")
for x in X:
   print(x)
print()
print("Matriz Y: ")
for y in Y:
   print(y)   
print()
print("Matriz resultante: ")
for r in result:
   print(r)
print( "------------------------------------")

print()
print( "Tiempos: " )
print( "------------------------------------")
# CPU - Informo tiempos, hilos y bloques.
tiempo_total = datetime.now() - tiempo_total
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print( "------------------------------------")

Resultados:
------------------------------------
Matriz X: 
[92]
[87]

Matriz Y: 
[13 97 63]

Matriz resultante: 
[1196 8924 5796]
[1131 8439 5481]
------------------------------------

Tiempos: 
------------------------------------
Tiempo CPU:  2.877 [ms]
------------------------------------


##![Important symbol](https://drive.google.com/uc?id=1AWRLAqeaqi7SG7PHyOVywZRuMDK9Z2_s)**Importante:** Debe cambiar de entorno de ejecución a GPU para poder ejecutar el siguiente desarrollo.

##3.2 Desarrollo - GPU

In [30]:
# --------------------------------------------
#@title 3.2.1 Parámetros de ejecución { vertical-output: true }

Filas_Matriz_A =   3#@param {type: "number"}
Columnas_Matriz_B = 5#@param {type: "number"}
Filas_B_Columnas_A = 1#@param {type: "number"}
# --------------------------------------------

# --------------------------------------------
# Valido número de ingreso por el usuario
if (Filas_B_Columnas_A <= 0 or Filas_Matriz_A < 0 or Columnas_Matriz_B < 0 ):
  raise Exception("Por favor, no ingrese números negativos. El atributo Filas_B_Columnas_A no puede ser igual a 0") 
# --------------------------------------------

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

import numpy as np
import time
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

from datetime import datetime
tiempo_total = datetime.now()

n = 4
ni = np.int32(n)

# CPU - Defino la memoria de los vectores en cpu.
a = np.random.randint(0, 100, (n, n))
a = a.astype(np.int32())

print("Matriz X: ")
for aa in a:
   print(aa)

b = np.random.randint(0, 100, (n, n))
b = b.astype(np.int32)

print("Matriz Y: ")
for bb in b:
   print(bb)

c = np.zeros((n,n), dtype=int)
c = c.astype(np.int32)

# CPU - reservo la memoria GPU.
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
c_gpu = cuda.mem_alloc(c.nbytes)

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

module = SourceModule("""
__global__ void matmul(int n, const float *A, const float *B, float *C){

  int tx = threadIdx.x;
  int ty = threadIdx.y;

  int bx = blockIdx.x;
  int by = blockIdx.y;

  int row = by*blockDim.y + ty;
  int col = bx*blockDim.x + tx;

  //Validamos que los hilos que no se encuentren dentro de las posiciones de memoria de las matrices no realizen ninguna tarea.
  if(row < n && col < n){
    float val = 0.0;
    for(int i=0; i<n; ++i){
      val += A[row*n + i]*B[n*i + col];
    }
    C[row*n + col] = val;
  }
}

""") 

# CPU - Genero la función kernel.
kernel = module.get_function("matmul")
tiempo_gpu = datetime.now()

dim_hilo_x = 16
dim_bloque_x = np.int( (4+dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 16
dim_bloque_y = np.int( (4+dim_hilo_y-1) / dim_hilo_y )

# GPU - Ejecuta el kernel.
kernel(np.int32(ni), a_gpu, b_gpu, c_gpu, block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1));
tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh(c, c_gpu)

print( "Resultados:" )
print( "------------------------------------")

print (c)

print( "------------------------------------")

tiempo_total = datetime.now() - tiempo_total

print()
print( "Tiempos: " )
print( "------------------------------------")
# TODO - Informo tiempos, hilos y bloques.
print( "Thread x: ", dim_hilo_x, ", Bloque x:", dim_bloque_x )
print( "Thread y: ", dim_hilo_y, ", Bloque y:", dim_bloque_y )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]") 
print( "------------------------------------")


Matriz X: 
[62 41 45 38]
[74 41 16 44]
[33 67  8 59]
[59 31 54 54]
Matriz Y: 
[85 86 84 13]
[14 44 22  4]
[81 10 35 79]
[30 36 76 21]
Thread x:  16 , Bloque x: 1
Thread y:  16 , Bloque y: 1
Resultados:
------------------------------------
------------------------------------

Tiempos: 
------------------------------------
Thread x:  256 , Bloque x: 1
Tiempo CPU:  5.134 [ms]
Tiempo GPU:  0.686 [ms]
------------------------------------


#4 Tabla de pasos

##4.1 Tabla de pasos - CPU

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura de los parametros necesarios para la generación de las matrices.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  raise Exception()     | Lanza una exception.
CPU      |  np.random.randint() | Inicializa las matrices con valores random de tipo entero según los parametros recibidos.
CPU      |  np.zeros()      | Inicializa la matriz que contendrá el resultado según los parámetros ingresados con el valor 0 en sus posiciones.
CPU      |  range()               | Genera una secuencia de valores enteros según los valores que recibe.
CPU      |  print()               | Informo los resultados.

##4.2 Tabla de pasos - GPU

#5 Conclusiones

[1] Las matrices generadas, que son poseen dos dimensiones, se almacenan en memoria en una sola dimensión siendo posible obtener el valor de una matriz como si fuese un vector realizando un sencillo cálculo para obtener la posición de memoria de la matriz y el valor que este contiene. Este hecho, hace que no tengamos que realizar varios ciclos en Cuda para poder acceder al valor de la matriz como si realizamos en Python.

[2] Cada hilo será el encargado de calcular el valor de uno de los valores de la matriz resultante, esto quiere decir que se generarán tantos hilos como posiciones posea la matriz resultante.

Como vemos, a diferencia del ejercicio 1 donde se debían sincronizar los hilos para que puedan realizar un ordenamiento óptimo, este tipo de algoritmo no posee tal desventaja ya que puede calcular cada valor de la matriz resultante sin necesidad de una sincronización con todos los demás. Esto le asigna a la implementación de este algoritmo con la ayuda del GPU muchas ventajas en relación a una versión que solo utiliza CPU.




#6 Referencias



*   [1] https://www.tutorialspoint.com/cuda/cuda_matrix_multiplication.htm
*   [2] https://www.fz-juelich.de/SharedDocs/Downloads/IAS/JSC/EN/slides/cuda/05-cuda-mm.pdf?__blob=publicationFile

